In [1]:
import pandas as pd
from datetime import date
import xlsxwriter

In [2]:
smds = ["Amy/YT Shih", "Ben Li", "Chen Ni/ Lele Zhou", "Hong Zeng", "Jennie Xie", "Licong He",
       "Perry Zhao/Yuemei Ding", "Xiao Chen", "Xiying Sheng", "Yinghua Zhang", "Yu Chen"]

In [3]:
file_path = r"C:\Users\carol\Downloads\81756455004 - Attendee Report.csv"
with open(file_path, 'r', encoding="utf-8") as file: 
    for i, line in enumerate(file):
        if line.startswith("Attendee Details"):
            breakline = i #line index where to break the csvfile
df_original = pd.read_csv(file_path, header=breakline+1, index_col=False)
df_original = df_original[["Attended","User Name (Original Name)","Join Time","Leave Time", "Time in Session (minutes)", 
         "Inviter", "Inviter's SMD"]]
print(f"Total {df_original.shape[0]} entries. The first 5 are showing below:")
df_original.head()

Total 214 entries. The first 5 are showing below:


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
0,No,Haibin,--,--,--,Hong Zeng,Hong Zeng
1,Yes,Xiaohui Xu,"Apr 08, 2021 19:28:22","Apr 08, 2021 20:55:32",88,Hong Hu,Perry Zhao/Yuemei Ding
2,No,Yingying,--,--,--,Hong Hu,Perry Zhao/Yuemei Ding
3,Yes,Cindy Huang,"Apr 08, 2021 19:15:49","Apr 08, 2021 19:48:07",33,Yuemei Ding,Perry Zhao/Yuemei Ding
4,Yes,Taiyi Li,"Apr 08, 2021 18:59:36","Apr 08, 2021 19:00:03",1,Perry,Perry Zhao/Yuemei Ding


In [4]:
df_smd_not_nan = df_original[~df_original["Inviter's SMD"].isna()]
df_smd_nan = df_original[df_original["Inviter's SMD"].isna()]

print(f"Total {df_smd_nan.shape[0]} attendees whoes Inviter's SMD is NaN. The first 5 are showing below:")
df_smd_nan.head()

Total 26 attendees whoes Inviter's SMD is NaN. The first 5 are showing below:


,Attended,User Name (Original Name),Join Time,Leave Time,Time in Session (minutes),Inviter,Inviter's SMD
5,Yes,Taiyi,"Apr 08, 2021 19:00:48","Apr 08, 2021 20:40:54",101,NaN,NaN
12,Yes,Guo Chen,"Apr 08, 2021 19:07:24","Apr 08, 2021 19:14:35",8,NaN,NaN
13,Yes,Guo Chen,"Apr 08, 2021 19:45:04","Apr 08, 2021 20:13:30",29,NaN,NaN
22,Yes,Amy Tian,"Apr 08, 2021 18:52:21","Apr 08, 2021 20:55:32",124,NaN,NaN
30,Yes,John Zhang,"Apr 08, 2021 19:09:20","Apr 08, 2021 19:09:45",1,NaN,NaN


In [5]:
#populate the NaN SMD based on User Name that appears before

true_nan_index = [] #those index of true nan: SMD is NaN and NoWay of filling in because it only appears once with SMD NaN
for i in df_smd_nan.index:
    user_name = df_smd_nan.loc[i]["User Name (Original Name)"]
    record_in_df_smd_not_nan = df_smd_not_nan[df_smd_not_nan["User Name (Original Name)"]==user_name]
    if record_in_df_smd_not_nan.shape[0] == 0:
        true_nan_index.append(i)
    elif record_in_df_smd_not_nan.shape[0] > 1:
        record_in_df_smd_not_nan["Join Time"] = pd.to_datetime(record_in_df_smd_not_nan["Join Time"])
        earliest_record_index = record_in_df_smd_not_nan["Join Time"].sort_values().index.values[0]
        missing_smd = record_in_df_smd_not_nan.loc[earliest_record_index]["Inviter's SMD"]
        df_smd_nan.loc[i].fillna(value={"Inviter's SMD":missing_smd}, inplace=True)
    else: #record_in_df_smd_not_nan.shape[0] == 1
        missing_smd = record_in_df_smd_not_nan["Inviter's SMD"].values[0]
        df_smd_nan.loc[i].fillna(value={"Inviter's SMD":missing_smd}, inplace=True)

In [6]:
df_smd_comb = pd.concat((df_smd_not_nan, df_smd_nan),axis=0)
df_true_nan = df_original.loc[true_nan_index]

In [7]:
num_output_entries = 0
for name in smds:
    name_replaced = name.replace("/"," or ")
    if df_smd_comb[df_smd_comb["Inviter's SMD"]==name].shape[0] > 0:
        file_name = f"{name_replaced}_{date.today().strftime('%m-%d-%Y')}_Attendee Report.xlsx"
        workbook = xlsxwriter.Workbook(file_name)
        worksheet = workbook.add_worksheet()
        worksheet.write('A1', 'Attendee Details')
        worksheet.write('A3', 'Attended')
        worksheet.write('B3', 'User Name (Original Name)')
        worksheet.write('C3', 'Join Time')
        worksheet.write('D3', 'Leave Time')
        worksheet.write('E3', 'Time in Session (minutes)')
        worksheet.write('F3', 'Inviter')
        worksheet.write('G3', "Inviter's SMD")
        
        df_name = df_smd_comb[df_smd_comb["Inviter's SMD"]==name]
        df_name['Time in Session (minutes)'] = pd.to_numeric(df_name['Time in Session (minutes)'], errors='coerce')
        df_name.sort_values(by='Time in Session (minutes)', ascending=False, inplace=True)
        df_name.reset_index(drop=True, inplace=True)
        df_name.fillna("-", inplace=True)
        for i, row in df_name.iterrows():
            worksheet.write(f'A{i+4}', row['Attended'])
            worksheet.write(f'B{i+4}', row['User Name (Original Name)'])
            worksheet.write(f'C{i+4}', row['Join Time'])
            worksheet.write(f'D{i+4}', row['Leave Time'])
            worksheet.write(f'E{i+4}', row['Time in Session (minutes)'])
            worksheet.write(f'F{i+4}', row['Inviter'])
            worksheet.write(f'G{i+4}', row["Inviter's SMD"])
            num_output_entries += 1
        workbook.close()

<ipython-input-7-d5500dfd2f06>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_name['Time in Session (minutes)'] = pd.to_numeric(df_name['Time in Session (minutes)'], errors='coerce')
<ipython-input-7-d5500dfd2f06>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_name.sort_values(by='Time in Session (minutes)', ascending=False, inplace=True)
C:\Users\carol\anaconda3\lib\site-packages\pandas\core\frame.py:4317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [8]:
#write others 
df_true_nan.to_excel(f"Others_{date.today().strftime('%m-%d-%Y')}_Attendee Report.xlsx", index=False)

In [9]:
print(f"The total number of output entries without true Nans are {num_output_entries}.")
print(f"The total number of true Nans are {len(true_nan_index)}.")
print(f"The total number of the above is {num_output_entries + len(true_nan_index)}.")
print(f"The total number of original records are {df_original.shape[0]}.")

The total number of output entries without true Nans are 209.
The total number of true Nans are 5.
The total number of the above is 214.
The total number of original records are 214.
